In [1]:
import importlib
import imageio.v2 as imageio
from patient import patient
import glob
import cupy as cp

# https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/6ACUZJ

from tqdm.notebook import tqdm

vol = "4409575"

# print(vol)
p = patient.patient(name=vol, num_views=450)

p.generate_drr(cont=False)

imgs = [cp.asnumpy(img) for img in p.drr.img]
imageio.mimsave("save.gif", [imgs[0]])

DRR:   0%|          | 0/450 [00:00<?, ?it/s]

In [2]:
p.generate_fbp(load_all=False, cont=False)

In [3]:
import imageio.v2 as imageio
import cupy as cp
imageio.imsave("test3.png", cp.asnumpy(p.fbp.get(150)))

In [3]:
import math
import numpy as np

def filter(px):
    if px < 40:
        return -px
    elif px < 60:
        return - 2 * (px - 40) + 40

cutoff = 40
def f(img):
    newimg = img.astype("float64")
    for c in range(0, 61):
        newimg[cp.all(newimg < c)] = filter(c)
    newimg[newimg > cutoff] = (newimg[newimg > cutoff] - cp.average(newimg[newimg > cutoff])) / cp.std(newimg[newimg > cutoff]) * 28.069 + 202.102
    newimg[newimg < 0] = 0
    newimg[newimg > 255] = 255
    return newimg.astype("uint8")

for idx, img in enumerate(p.drr.img):
    p.drr.img[idx] = f(img)

imgs = [cp.asnumpy(img) for img in p.drr.img]
imageio.mimsave("save.gif", imgs)

In [21]:
import numpy as np
import hashlib
from perlin_noise import PerlinNoise

p.generate_fbp(load_all=False, cont=False)

import matplotlib.pyplot as plt


def circ(inside, outside, height, width):
    f = np.full((height, width), 1)
    i, o = (inside, outside)
    hh, hw = (height / 2, width / 2)
    for x in range(0, height):
        for y in range(0, width):
            dist = math.sqrt((x - hh) * (x - hh) + (y - hw) * (y - hw))
            if dist > i:
                f[x][y] = 0
    return f

def getnoise(octaves, intensity, height=500, seed=1):
    noise = PerlinNoise(octaves=octaves, seed=seed)
    xpix, ypix = 500, 500
    pic = [[noise([i/xpix, j/ypix]) for j in range(xpix)] for i in range(ypix)]

    pic = np.stack(pic)
    pic /= np.max(pic)
    pic *= circ(150, 0, 500, 500)
    pic *= intensity
    pic += 1

    for x in range (0, len(pic)):
        for y in range (0, len(pic[x])):
            pass

    return pic

s = "8945731"
seed = int(hashlib.sha1(s.encode("utf-8")).hexdigest(), 16)

smallnoise = getnoise(10, 0.1, seed=seed)
midnoise = getnoise(6, 0.125, seed=seed * 2)
bignoise = getnoise(2, 0.15, seed=seed * 3)
# plt.imshow(img * pic, cmap='gray')

imageio.imsave("test2.png", cp.asnumpy(p.fbp.get(200)) * smallnoise * midnoise * bignoise)
imageio.imsave("ct.png", cp.asnumpy(p.ct.img[200]))

Lossy conversion from float64 to uint8. Range [0.0, 272.17412465663386]. Convert image to uint8 prior to saving to suppress this warning.


In [9]:
imageio.imsave("test2.png", cp.asnumpy(p.get_equiv_fbp(300)))
# imageio.imsave("test2.png", cp.asnumpy(p.ct.img[150]))

KeyboardInterrupt: 